In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score


from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier

In [9]:
X_train = pd.read_csv('../data/participants_train.csv')
X_test = pd.read_csv('../data/participants_test.csv')
y_train = pd.read_csv('../data/train_winners.csv')
submission = pd.read_csv('../data/sample_submission.csv')
champion_mastery = pd.read_csv('../data/champion_mastery.csv')

In [10]:
X_train.head()

,matchId,teamId,participantId,summonerId,summonerLevel,championName,championId
0,0,100,1,0,303,Mordekaiser,82
1,0,100,2,1,616,Sylas,517
2,0,100,3,2,667,Lissandra,127
3,0,100,4,3,860,Caitlyn,51
4,0,100,5,4,325,Morgana,25


In [52]:
X_test_champion_mastery = pd.merge(X_train, champion_mastery, how='left', on=['summonerId', 'championId'], indicator=True)
X_test_champion_mastery

,matchId,teamId,participantId,summonerId,summonerLevel,championName,championId,championLevel,championPoints,chestGranted,tokensEarned,_merge
0,0,100,1,0,303,Mordekaiser,82,6.0,61890.0,True,3.0,both
1,0,100,2,1,616,Sylas,517,5.0,37664.0,True,2.0,both
2,0,100,3,2,667,Lissandra,127,6.0,40303.0,True,1.0,both
3,0,100,4,3,860,Caitlyn,51,6.0,96304.0,True,3.0,both
4,0,100,5,4,325,Morgana,25,7.0,608721.0,True,0.0,both
...,...,...,...,...,...,...,...,...,...,...,...,...
79995,7999,200,6,13979,595,Yorick,83,6.0,48037.0,False,1.0,both
79996,7999,200,7,39643,38,Volibear,106,1.0,135.0,False,0.0,both
79997,7999,200,8,5570,498,Anivia,34,7.0,49027.0,False,0.0,both
79998,7999,200,9,10228,733,Twitch,29,7.0,174089.0,True,0.0,both


In [54]:
match_df = X_test_champion_mastery.groupby('matchId')['tokensEarned'].sum().reset_index()
match_df

,matchId,tokensEarned
0,0,11.0
1,1,2.0
2,2,2.0
3,3,5.0
4,4,10.0
...,...,...
7995,7995,5.0
7996,7996,2.0
7997,7997,5.0
7998,7998,0.0


In [59]:
train_predictions = ( 
  X_test_champion_mastery
.sort_values(['championPoints'], ascending = [False])
.drop_duplicates('matchId')
.reset_index(drop = True)
)

train_predictions

,matchId,teamId,participantId,summonerId,summonerLevel,championName,championId,championLevel,championPoints,chestGranted,tokensEarned,_merge
0,6739,100,3,24310,777,Talon,91,7.0,7851299.0,True,0.0,both
1,7028,100,3,24310,786,Talon,91,7.0,7851299.0,True,0.0,both
2,3769,200,8,24310,788,Talon,91,7.0,7851299.0,True,0.0,both
3,3140,100,2,474,1290,Volibear,106,7.0,7370145.0,True,0.0,both
4,460,200,7,474,1293,Volibear,106,7.0,7370145.0,True,0.0,both
...,...,...,...,...,...,...,...,...,...,...,...,...
7995,2061,200,9,15632,676,Jinx,222,7.0,52251.0,False,0.0,both
7996,7649,100,3,38599,475,Lux,99,5.0,45836.0,True,1.0,both
7997,1970,200,7,15115,51,Kayn,141,6.0,43785.0,True,3.0,both
7998,3436,200,7,22754,328,Evelynn,28,6.0,40982.0,True,1.0,both


In [50]:
y_train

,matchId,winner
0,0,200
1,1,200
2,2,200
3,3,100
4,4,200
...,...,...
7995,7995,200
7996,7996,100
7997,7997,200
7998,7998,200


In [47]:
from sklearn.metrics import accuracy_score

In [60]:
accuracy_score(
    y_true = y_train['winner'],
    y_pred = train_predictions['teamId']
)

0.512125

### Try 2: